Convert a Binary File to a Grayscale Image

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def binary_to_image(binary_file, output_image, width=256):
    # Read binary file
    with open(binary_file, "rb") as f:
        data = f.read()
    
    # Convert binary data to numpy array
    byte_array = np.frombuffer(data, dtype=np.uint8)
    
    # Determine image dimensions
    height = len(byte_array) // width
    byte_array = byte_array[:height * width]  # Trim excess bytes
    image_array = byte_array.reshape((height, width))

    # Save as image
    plt.imsave(output_image, image_array, cmap='gray')
    print(f"Image saved as {output_image}")

# Example usage
binary_to_image("uploaded_models\spam_detection_model_Working.joblib", "uploaded_models\spam_detection_model_Working.png")


Image saved as uploaded_models\spam_detection_model_Working.png


Convert Binary File to RGB Image

In [ ]:
def binary_to_rgb_image(binary_file, output_image, width=256):
    with open(binary_file, "rb") as f:
        data = f.read()

    byte_array = np.frombuffer(data, dtype=np.uint8)
    height = len(byte_array) // (width * 3)  # 3 bytes per pixel (RGB)
    byte_array = byte_array[:height * width * 3]  # Trim excess
    image_array = byte_array.reshape((height, width, 3))

    plt.imsave(output_image, image_array)
    print(f"RGB Image saved as {output_image}")

# Example usage
binary_to_rgb_image("sample.bin", "output_rgb.png")


In [ ]:
import numpy as np
import cv2
import struct
import os

def binary_to_image(input_binary, output_image, width=256):
    # Read binary file
    with open(input_binary, "rb") as f:
        byte_data = f.read()

    original_size = len(byte_data)  # Store original file size

    # Pad data to fit into an image
    padding = (width - (len(byte_data) % width)) % width
    byte_data += b"\x00" * padding  # Zero-padding

    # Convert to numpy array
    byte_array = np.frombuffer(byte_data, dtype=np.uint8)

    # Reshape into a 2D image (grayscale)
    height = len(byte_array) // width
    image = byte_array.reshape((height, width))

    # Save original file size in the first row
    metadata = struct.pack("I", original_size)  # 4 bytes for file size
    image = np.vstack([np.frombuffer(metadata, dtype=np.uint8).reshape(1, -1), image])

    # Save as PNG
    cv2.imwrite(output_image, image)

    print(f"Image saved: {output_image} (Original Size: {original_size} bytes)")

# Example usage
binary_to_image("original.bin", "output_with_metadata.png")


In [ ]:
binary_to_image("original.bin", "output_with_metadata.png")
image_to_binary("output_with_metadata.png", "reconstructed.bin")

# Verify if the files are identical
with open("original.bin", "rb") as f1, open("reconstructed.bin", "rb") as f2:
    assert f1.read() == f2.read(), "Files do not match!"
print("Success! The reconstructed file matches the original.")


In [ ]:
Step 2: Convert Image → Binary (With Metadata Handling)

In [ ]:
def image_to_binary(image_file, output_binary):
    # Load image in grayscale
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

    if img is None:
        raise ValueError(f"Failed to load image: {image_file}")

    # Extract metadata (first row stores file size)
    metadata = img[0, :4]  # First 4 bytes contain the file size
    original_size = struct.unpack("I", metadata.tobytes())[0]

    # Flatten the image (excluding metadata row)
    byte_array = img[1:].flatten()

    # Trim padding
    byte_array = byte_array[:original_size]

    # Save as binary file
    with open(output_binary, "wb") as f:
        f.write(byte_array.tobytes())

    print(f"Reconstructed binary file saved as: {output_binary} (Original Size: {original_size} bytes)")

# Example usage
image_to_binary("output_with_metadata.png", "reconstructed.bin")


 Bulk Image Comparison Script

In [4]:
!pip install opencv-python 
# numpy pandas scikit-image matplotlib

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.5/39.5 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.5/39.5 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.8/39.5 MB 714

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt

# Folders containing images for comparison
IMAGE_FOLDER = "./uploaded_models"
DIFF_OUTPUT_FOLDER = "./diff_images"
RESULTS_CSV = "comparison_results.csv"

# Ensure output directory exists
os.makedirs(DIFF_OUTPUT_FOLDER, exist_ok=True)

# Get list of images
image_files = sorted([f for f in os.listdir(IMAGE_FOLDER) if f.endswith(".png")])

# Store results
comparison_results = []

# Compare all pairs of images
for i in range(len(image_files)):
    for j in range(i + 1, len(image_files)):
        img1_path = os.path.join(IMAGE_FOLDER, image_files[i])
        img2_path = os.path.join(IMAGE_FOLDER, image_files[j])
        diff_output_path = os.path.join(DIFF_OUTPUT_FOLDER, f"diff_{image_files[i]}_{image_files[j]}.png")

        # Load images in grayscale
        img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

        # Skip if sizes don't match
        if img1.shape != img2.shape:
            print(f"Skipping {image_files[i]} & {image_files[j]} (Different sizes)")
            continue

        # Compute absolute pixel difference
        diff = cv2.absdiff(img1, img2)
        cv2.imwrite(diff_output_path, diff)

        # Compute SSIM score
        ssim_score, _ = ssim(img1, img2, full=True)

        # Compute histogram similarity
        hist1 = cv2.calcHist([img1], [0], None, [256], [0, 256])
        hist2 = cv2.calcHist([img2], [0], None, [256], [0, 256])
        hist_similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

        # Store result
        comparison_results.append({
            "Image 1": image_files[i],
            "Image 2": image_files[j],
            "SSIM Score": round(ssim_score, 4),
            "Histogram Similarity": round(hist_similarity, 4),
            "Difference Image": diff_output_path
        })

        print(f"Compared {image_files[i]} vs {image_files[j]} | SSIM: {ssim_score:.4f} | Hist: {hist_similarity:.4f}")

# Save results to CSV
df = pd.DataFrame(comparison_results)
df.to_csv(RESULTS_CSV, index=False)
print(f"Comparison results saved to {RESULTS_CSV}")


ImportError: DLL load failed while importing mtrand: An Application Control policy has blocked this file.